# PyDVL - Shapley for data valuation

This notebook introduces Shapley methods for the computation of data value using pyDVL.

In order to show the practical advantages of shapley pydvl implementation, we will predict the popularity of songs in the "Top Hits Spotify from 2000-2019" dataset. While doing so, we will highlight how data valuation can help boost the performance of your models.

Here, all the library main entry-points will be briefly described. We will also show the advantages of this library compared to vanilla data-Shapley implementations, like runtime optimization for large datasets and models.

Let's start with some imports

In [1]:
%load_ext autoreload

To load the dataset, we will use the load_spotify_dataset method. Internally, the method will load data on songs published after 2014, use 30% of data for test, and another 30% of the remaining data for validation. Then, the method will return train, validation and test data as lists of the shape [X_input, Y_label]

In [2]:
from valuation.utils import load_spotify_dataset
train_data, val_data, test_data = load_spotify_dataset(val_size=0.3, test_size=0.3, target_column='popularity')
train_data[0].head()

/Users/fabio/miniconda3/envs/data_shapley/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,artist,song,duration_ms,explicit,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
1782,Justin Bieber,Friends (with BloodPop®),189466,False,2017,0.744,0.739,8,-5.350,1,0.0387,0.00459,0.000000,0.3060,0.649,104.990,14
1693,Bryson Tiller,Exchange,194613,True,2015,0.525,0.433,6,-10.598,1,0.1850,0.10700,0.000000,0.1350,0.276,160.108,10
1512,The Neighbourhood,Daddy Issues,260173,False,2015,0.588,0.521,10,-9.461,1,0.0329,0.06780,0.149000,0.1230,0.337,85.012,26
1938,Drake,Money In The Grave (Drake ft. Rick Ross),205426,True,2019,0.831,0.502,10,-4.045,0,0.0460,0.10100,0.000000,0.1220,0.101,100.541,10
1665,Fifth Harmony,Work from Home (feat. Ty Dolla $ign),214480,False,2016,0.803,0.585,8,-5.861,1,0.0432,0.10300,0.000004,0.0644,0.593,105.017,14


The dataset has many high level feature, some quite intuitive ('duration_ms' or 'tempo'), and other a bit more cryptic ('valence'?). If you want more information on each feature, you can find it on [this webpage](https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019?resource=download).
In our analysis, we will use all the columns, excluding 'artist' and 'song', to predict the 'popularity' of each song. We will nonetheless keep the information on song and artist in the train set in a separate pandas Series, for future reference.

In [3]:
song_name = train_data[0]['song']
artist = train_data[0]['artist']
train_data[0] = train_data[0].drop(['song', 'artist'], axis=1)
test_data[0] = test_data[0].drop(['song', 'artist'], axis=1)
val_data[0] = val_data[0].drop(['song', 'artist'], axis=1)

Input and label data need to be stored in a Dataset object, a pyDVL specific tool.

In [4]:
from valuation.utils import Dataset
dataset = Dataset(*train_data, *val_data)

The calculation of Shapley coefficients is very computationally expensive because it needs to go through several subsets of the original input dataset before converging. For this reason, the PyDVL library implements techniques to speed up the calculation, both caching intermediate results and allowing to group data to calculate Shapley values on groups instead of single-datapoints. From a Dataset object, you can just do the following.

In [5]:
from valuation.utils import GroupedDataset
grouped_dataset = GroupedDataset.from_dataset(dataset, artist)

And now we can finally calculate the contribution of each datapoint to the modle performance!

As model, we will use a GradientBoostingRegressor, but any model from sklearn, xgboost and lightgbm works. More precisely, any model that has a fit and predict method should run without issues.

Note: Make sure to restart (or simply start if it is not already running) your memcache. See our [documentation](https://appliedai-initiative.github.io/valuation/install.html) for details.

In [6]:
from valuation.shapley import get_shapley_values
from valuation.utils import Utility
from sklearn.ensemble import GradientBoostingRegressor

utility = Utility(
    model=GradientBoostingRegressor(n_estimators=3),
    data=grouped_dataset,
    scoring='neg_mean_absolute_error',
    # TODO: This should be set parametrically
    enable_cache=False,
)
dvl_df = get_shapley_values(utility, iterations_per_job=50, num_jobs=20)

KeyboardInterrupt: 

As you can see, the first object that we have defined (utility) holds all the information on the dataset, the model and the scoring methods. Within utility, the PyDVL library uses a cache to speed up calculation of the Shapley coefficients.
The second method, get_shapley_values, calculates the actual Shapley values and manages parallelization. Let's take a look at the returned dataframe.

In [ ]:
dvl_df.head()

The first thing to notice is that the dvl_df dataframe is sorted in ascending order of shapley values. The data_key columns holds the labels for each data group: in this case, since the groups correspond to artists, it coincides with artist names. The second column corresponds to the shapley data valuation score, and the third to its (approximate) standard deviation.

A better way to analyse the results is through a plot. In the next cell we will take the 30 datapoints with lowest score and plot them with errorbars

In [ ]:
from valuation.utils import plot_shapley
low_dvl = dvl_df.iloc[:30]
fig = plot_shapley(low_dvl, figsize=(20, 4), title='Artists with low data valuation scores', xlabel='artist', ylabel='shapley_pydvl')
fig.show()

As you can see, there are a lot of points which give negative shapley score, meaning that they tend to decrease the total score of the model when present in the training set! What happens if we remove it? In the next cell we will create a new training set which excludes the 30 points with lowest scores.

In [ ]:
low_dvl_artists = dvl_df.iloc[:30].data_key.to_list()
artist_filter = ~artist.isin(low_dvl_artists)
X_train_good_dvl = train_data[0][artist_filter]
y_train_good_dvl = train_data[0][artist_filter]

Now we will use this cleaned dataset to train a full GradientBoostingRegressor and compare its mean absolute error to the model which uses the full dataset. Notice that the score now is calculated using the test set, while for calculating the shapley values we were using the validation set.

In [ ]:
from sklearn.metrics import mean_absolute_error
full_model = GradientBoostingRegressor(n_estimators=3).fit(X_train_good_dvl, y_train_good_dvl)
mean_absolute_error(full_model.predict(test_data[0]), test_data[1])

In [ ]:
full_model = GradientBoostingRegressor(n_estimators=3).fit(train_data[0], train_data[1])
mean_absolute_error(full_model.predict(test_data[0]), test_data[1])

The score has improved by more than 15%! This is quite an important result, as it shows a self-consistent process to improve the performance of a model by excluding datapoints from its training set.

## Evaluation on anomalous data

Another interesting test is to corrupt some of the data and monitor how their valuation score changes. To do this, we will take one of the authors with the highest score and set all its popularity to 0.

In [7]:
high_dvl = dvl_df.iloc[-30:]
fig = plot_shapley(high_dvl, figsize=(20, 4), title='Artists with high data valuation scores', xlabel='artist', ylabel='shapley values')
fig.show()

NameError: name 'dval_df' is not defined

From the plot above, we can see that Rihanna has one of the highest scores. Let's now take all the train labels related to her, set the score to 0 and re-calculate the data valuation scores. Notice that I must use a new name for the dataset, because I need to create another cache. Reusing the old one would be a mistake since modifying even just one label changes all the shapley values.

In [8]:
train_data[0].loc[artist == 'Rihanna'] = 0
dataset = Dataset(*train_data, *val_data)
grouped_dataset = GroupedDataset.from_dataset(dataset, artist)
utility = Utility(
        model=GradientBoostingRegressor(n_estimators=3), data=grouped_dataset, scoring='neg_mean_absolute_error', enable_cache=True,
    )
dvl_df = get_shapley_values(utility, iterations_per_job=50, num_jobs=20)

(b'|\x01s\x04d\x01S\x00t\x00|\x00j\x01|\x00j\x02\x83\x02}\x02|\x00j\x03\xa0\x04t\x05|\x01\x83\x01\xa1\x01\\\x02}\x03}\x04z\x15|\x00j\x01\xa0\x06|\x03|\x04\xa1\x02\x01\x00t\x07|\x02|\x00j\x01|\x00j\x03j\x08|\x00j\x03j\t\x83\x03\x83\x01W\x00S\x00\x04\x00t\nyK\x01\x00}\x05\x01\x00z\x15|\x00j\x0brEt\x0c\xa0\rt\x0e|\x05\x83\x01\xa1\x01\x01\x00|\x00j\x0fW\x00\x06\x00Y\x00d\x02}\x05~\x05S\x00|\x05\x82\x01d\x02}\x05~\x05w\x01w\x00', ('Fits the model on a subset of the training data and scores it on the\n        test data. If the object is constructed with cache_size > 0, results are\n        memoized to avoid duplicate computation. This is useful in particular\n        when computing utilities of permutations of indices.\n\n        :param indices: a subset of indices from data.x_train.index. The type\n         must be hashable for the caching to work, e.g. wrap the argument with\n         `frozenset` (rather than `tuple` since order should not matter)\n\n        :return: 0 if no indices are pa

KeyboardInterrupt: 

Let's now take the low scoring artists and plot the results

In [ ]:
low_dvl = dvl_df.iloc[:30]
fig = plot_shapley(low_dvl, figsize=(20, 4), title='Artists with low data valuation scores', xlabel='artist', ylabel='shapley values')
fig.show()

And Rihanna (our anomalous data group) has moved from top contributor to having negative impact on the performance of the model, as expected!